# American Express - Default Prediction
Predict if a customer will default in the future

Whether out at a restaurant or buying tickets to a concert, modern life counts on the convenience of a credit card to make daily purchases. It saves us from carrying large amounts of cash and also can advance a full purchase that can be paid over time. How do card issuers know we’ll pay back what we charge? That’s a complex problem with many existing solutions—and even more potential improvements, to be explored in this competition.

Credit default prediction is central to managing risk in a consumer lending business. Credit default prediction allows lenders to optimize lending decisions, which leads to a better customer experience and sound business economics. Current models exist to help manage risk. But it's possible to create better models that can outperform those currently in use.

American Express is a globally integrated payments company. The largest payment card issuer in the world, they provide customers with access to products, insights, and experiences that enrich lives and build business success.

In this competition, you’ll apply your machine learning skills to predict credit default. Specifically, you will leverage an industrial scale data set to build a machine learning model that challenges the current model in production. Training, validation, and testing datasets include time-series behavioral data and anonymized customer profile information. You're free to explore any technique to create the most powerful model, from creating features to using the data in a more organic way within a model.

If successful, you'll help create a better customer experience for cardholders by making it easier to be approved for a credit card. Top solutions could challenge the credit default prediction model used by the world's largest payment card issuer—earning you cash prizes, the opportunity to interview with American Express, and potentially a rewarding new career.

## Data Description

The objective of this competition is to predict the probability that a customer does not pay back their credit card balance amount in the future based on their monthly customer profile. The target binary variable is calculated by observing 18 months performance window after the latest credit card statement, and if the customer does not pay due amount in 120 days after their latest statement date it is considered a default event.

The dataset contains aggregated profile features for each customer at each statement date. Features are anonymized and normalized, and fall into the following general categories:

*   D\_\* = Delinquency variables
*   S\_\* = Spend variables
*   P\_\* = Payment variables
*   B\_\* = Balance variables
*   R\_\* = Risk variables

with the following features being categorical:

`['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']`

Your task is to predict, for each `customer_ID`, the probability of a future payment default (`target = 1`).

Note that the negative class has been subsampled for this dataset at 5%, and thus receives a 20x weighting in the scoring metric.

Files
-----

*   **train\_data.csv** - training data with multiple statement dates per `customer_ID`
*   **train\_labels.csv** - `target` label for each `customer_ID`
*   **test\_data.csv** - corresponding test data; your objective is to predict the `target` label for each `customer_ID`
*   **sample\_submission.csv** - a sample submission file in the correct format

Link: https://www.kaggle.com/competitions/amex-default-prediction/overview

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from tqdm.notebook import tqdm
import tsfresh
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score

In [2]:
from utils import amex_metric

In [3]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [4]:
le = preprocessing.LabelEncoder()

<IPython.core.display.Javascript object>

In [5]:
df = pd.read_csv(
    "../../data/amex-default-prediction/sample_train_data_30k.csv", delimiter="\t"
).set_index("customer_ID")
df

,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
customer_ID,,,,,,,,,,,,,,,,,,,,,
567bb352c1ec9fe043c64dd1c5e075189c30612b456a76cbeaf0cb34341d3a4f,2017-12-29,0.392097,0.002878,0.040471,0.810208,0.004398,0.244489,0.004483,0.007960,0.270353,...,NaN,NaN,NaN,0.002539,NaN,NaN,NaN,0.006251,NaN,1
567bb352c1ec9fe043c64dd1c5e075189c30612b456a76cbeaf0cb34341d3a4f,2018-01-19,0.391909,0.036332,0.057248,0.816525,0.006929,0.234017,0.005245,0.001577,0.269885,...,NaN,NaN,0.004141,0.000521,0.005122,NaN,0.008661,0.004245,0.004008,1
567bb352c1ec9fe043c64dd1c5e075189c30612b456a76cbeaf0cb34341d3a4f,2018-02-08,0.361950,0.623582,0.089909,0.818877,0.007582,0.332134,0.002409,0.002631,0.274014,...,NaN,NaN,0.007783,0.006298,0.008711,NaN,0.006385,0.006506,0.002321,1
567bb352c1ec9fe043c64dd1c5e075189c30612b456a76cbeaf0cb34341d3a4f,2018-03-28,0.240591,0.300403,0.106174,0.054358,0.508895,0.451140,0.007622,0.169807,0.269052,...,NaN,NaN,0.005840,0.006598,0.003271,NaN,0.001816,0.003071,0.002218,1
443132d9591d20ee365043b961a8d3d67d96daab5d75469b03d4bd64a4f31b8f,2017-03-18,0.275838,0.648851,0.035793,0.841584,0.751161,0.073957,0.538993,0.026267,NaN,...,NaN,NaN,0.000245,0.000904,0.002443,NaN,0.003759,0.000124,0.002175,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9a7e60eee07bcb770a14e8b9dca634b62e2eda22947dd01fece4880fe9a6200c,2017-11-11,0.409957,0.003579,0.568713,0.011909,0.006221,0.132613,0.009133,0.439427,0.177524,...,NaN,NaN,0.005938,0.005980,0.000688,NaN,0.005720,0.004623,0.001788,1
9a7e60eee07bcb770a14e8b9dca634b62e2eda22947dd01fece4880fe9a6200c,2017-12-16,0.393445,0.002969,0.572830,0.004148,0.005036,0.129955,0.001435,0.437734,0.174953,...,NaN,NaN,0.005850,0.000524,0.008980,NaN,0.007779,0.006147,0.001454,1
9a7e60eee07bcb770a14e8b9dca634b62e2eda22947dd01fece4880fe9a6200c,2018-01-19,0.321152,0.035907,0.599499,0.002268,0.004242,0.130136,0.001243,0.448089,0.170339,...,0.003078,0.506428,0.003302,0.003661,0.005257,NaN,0.006914,0.007320,0.006374,1


<IPython.core.display.Javascript object>

In [6]:
len(df.groupby("customer_ID"))

29136

<IPython.core.display.Javascript object>

# Feature engineering

In [7]:
df.sort_values(["customer_ID", "S_2"], inplace=True)

<IPython.core.display.Javascript object>

In [8]:
df.drop("S_2", axis=1, inplace=True)

<IPython.core.display.Javascript object>

## Categorical

In [9]:
cat_columns = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68",
]

<IPython.core.display.Javascript object>

In [10]:
for cat_column in tqdm(cat_columns):
    df[cat_column] = le.fit_transform(df[cat_column])

categorical_df = df[cat_columns].groupby("customer_ID").max().copy()
categorical_df

  0%|          | 0/11 [00:00<?, ?it/s]

,B_30,B_38,D_114,D_116,D_117,D_120,D_126,D_63,D_64,D_66,D_68
customer_ID,,,,,,,,,,,
000201146e53cacdde1c7e9d29f4d3c46fd4d9231a3744aa39fb9c6afa79b708,1,4,1,0,6,0,2,1,1,1,6
00055add5eaee481b27e40af3f47b8b24e82c1e550f6ab010000e7685692f281,0,0,0,0,3,1,1,1,3,2,4
0005e52a3fa31b7eed49ceb576f011433ee2578833cd3f9d51c9dd9448a198ff,2,3,2,2,7,2,3,1,4,2,7
0008c2f297e1b00bf567c0d2c25f3e3b356f9a3088d2bf47aaaa724d26df8787,1,6,1,0,4,1,2,1,1,2,6
000ee46c042bfab551c28d92c93969f8a3539fe1e9fc9cd2d2e188f838f7d8ba,0,1,1,0,0,0,2,1,3,2,5
...,...,...,...,...,...,...,...,...,...,...,...
fff5d7b60546129d6a8329f6d7ae6fa18e5ebae6e2ce445f96eb933f4d62e347,0,1,2,2,7,2,3,2,4,2,7
fff7c9afcb6185ebe6152587453d9bd1eb04c458005d64c6afba25f32faa51f2,0,1,1,0,5,0,2,2,4,2,6
fffce0cbbd3dc853a970145cc1ebd44c716af633214cd0b54d18b01fbd31b626,1,4,1,0,0,0,2,1,3,2,5


<IPython.core.display.Javascript object>

## Numerical

In [11]:
num_columns = list(set(df.drop("target", axis=1).columns) - set(cat_columns))
len(num_columns)

177

<IPython.core.display.Javascript object>

In [12]:
df1 = df[num_columns].copy()
df1[num_columns] = df1[num_columns].fillna(0)
df1

,R_24,D_128,B_15,B_18,D_143,S_15,R_12,R_23,R_21,D_81,...,D_118,D_122,D_52,R_6,D_138,D_104,D_47,R_9,B_27,D_106
customer_ID,,,,,,,,,,,,,,,,,,,,,
000201146e53cacdde1c7e9d29f4d3c46fd4d9231a3744aa39fb9c6afa79b708,0.009399,1.001423,0.007800,0.482771,0.008340,0.108498,1.000623,0.006281,0.001485,0.001664,...,0.275691,0.288140,0.288595,0.009311,0.0,0.956719,0.354323,0.0,0.004468,0.0
000201146e53cacdde1c7e9d29f4d3c46fd4d9231a3744aa39fb9c6afa79b708,0.009699,1.007576,0.002514,0.477936,0.008119,0.205234,1.006720,0.001686,0.008436,0.004318,...,0.276610,0.294517,0.289621,0.006685,0.0,0.949323,0.350338,0.0,0.001176,0.0
000201146e53cacdde1c7e9d29f4d3c46fd4d9231a3744aa39fb9c6afa79b708,0.003843,1.001856,0.003168,0.055250,0.005152,0.202382,1.004813,0.006082,0.001760,0.003415,...,0.281240,0.295019,0.290904,0.006592,0.0,0.950712,0.351637,0.0,0.007254,0.0
000201146e53cacdde1c7e9d29f4d3c46fd4d9231a3744aa39fb9c6afa79b708,0.008246,1.007357,0.004135,0.054491,0.003542,0.203058,1.008179,0.009860,0.004737,0.007226,...,0.281829,0.290225,0.291763,0.003875,0.0,0.957371,0.360516,0.0,0.006132,0.0
000201146e53cacdde1c7e9d29f4d3c46fd4d9231a3744aa39fb9c6afa79b708,0.002318,1.001103,0.003647,0.312924,0.000192,0.101600,1.007445,0.004935,0.001289,0.008658,...,0.280681,0.287595,0.287962,0.002749,0.0,0.955581,0.360855,0.0,0.002881,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffec7d7e1ca804c86f1ffdaac389c33f8039ed35bf412b12d2e3548e49d54fa,0.001910,0.000755,0.007673,1.006863,0.001862,0.508130,1.006457,0.005899,0.001072,0.002972,...,0.002120,0.290743,1.009317,0.003231,0.0,0.008867,0.357803,0.0,0.008825,0.0
fffec7d7e1ca804c86f1ffdaac389c33f8039ed35bf412b12d2e3548e49d54fa,0.007407,0.009188,0.003287,1.004954,0.003419,0.508009,1.001680,0.004952,0.009692,0.007425,...,0.004081,0.285720,1.002162,0.007641,0.0,0.006933,0.363031,0.0,0.005575,0.0
fffec7d7e1ca804c86f1ffdaac389c33f8039ed35bf412b12d2e3548e49d54fa,0.000109,0.006086,0.002357,1.007119,0.003603,0.507008,1.000973,0.000909,0.007734,0.009112,...,0.002444,0.291161,1.004501,0.003109,0.0,0.000963,0.359237,0.0,0.007944,0.0


<IPython.core.display.Javascript object>

In [13]:
df2 = tsfresh.extract_features(
    df1.reset_index(),
    column_id="customer_ID",
    default_fc_parameters=tsfresh.feature_extraction.MinimalFCParameters(),
)
df2.shape

Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 30/30 [09:17<00:00, 18.59s/it]


(29136, 1770)

<IPython.core.display.Javascript object>

In [14]:
df2

,R_24__sum_values,R_24__median,R_24__mean,R_24__length,R_24__standard_deviation,R_24__variance,R_24__root_mean_square,R_24__maximum,R_24__absolute_maximum,R_24__minimum,...,D_106__sum_values,D_106__median,D_106__mean,D_106__length,D_106__standard_deviation,D_106__variance,D_106__root_mean_square,D_106__maximum,D_106__absolute_maximum,D_106__minimum
000201146e53cacdde1c7e9d29f4d3c46fd4d9231a3744aa39fb9c6afa79b708,0.067303,0.004525,0.005177,13.0,0.002827,7.991826e-06,0.005899,0.009699,0.009699,0.000649,...,0.000000,0.000000,0.000000,13.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
00055add5eaee481b27e40af3f47b8b24e82c1e550f6ab010000e7685692f281,0.072608,0.005156,0.005585,13.0,0.002429,5.902278e-06,0.006091,0.009010,0.009010,0.000410,...,0.000000,0.000000,0.000000,13.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0005e52a3fa31b7eed49ceb576f011433ee2578833cd3f9d51c9dd9448a198ff,0.033194,0.004326,0.004149,8.0,0.002699,7.284470e-06,0.004950,0.008165,0.008165,0.000209,...,0.000000,0.000000,0.000000,8.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0008c2f297e1b00bf567c0d2c25f3e3b356f9a3088d2bf47aaaa724d26df8787,0.067262,0.004502,0.005174,13.0,0.002472,6.109416e-06,0.005734,0.009218,0.009218,0.001211,...,0.000000,0.000000,0.000000,13.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
000ee46c042bfab551c28d92c93969f8a3539fe1e9fc9cd2d2e188f838f7d8ba,0.066875,0.005031,0.005144,13.0,0.002926,8.561354e-06,0.005918,0.009853,0.009853,0.000882,...,0.000000,0.000000,0.000000,13.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff5d7b60546129d6a8329f6d7ae6fa18e5ebae6e2ce445f96eb933f4d62e347,0.063645,0.004536,0.004896,13.0,0.002560,6.554579e-06,0.005525,0.009860,0.009860,0.000167,...,0.000000,0.000000,0.000000,13.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
fff7c9afcb6185ebe6152587453d9bd1eb04c458005d64c6afba25f32faa51f2,0.059593,0.005046,0.004584,13.0,0.003016,9.098401e-06,0.005487,0.009918,0.009918,0.000516,...,0.000000,0.000000,0.000000,13.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
fffce0cbbd3dc853a970145cc1ebd44c716af633214cd0b54d18b01fbd31b626,0.048996,0.002628,0.003769,13.0,0.002673,7.143774e-06,0.004620,0.008724,0.008724,0.000061,...,1.194719,0.092673,0.091901,13.0,0.002604,0.000007,0.091938,0.095634,0.095634,0.087594
fffec7d7e1ca804c86f1ffdaac389c33f8039ed35bf412b12d2e3548e49d54fa,0.029304,0.005143,0.004186,7.0,0.002458,6.043732e-06,0.004855,0.007407,0.007407,0.000109,...,0.000000,0.000000,0.000000,7.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


limit_output extension: Maximum message size of 10000 exceeded with 16798 characters

# Data Preprocessing

In [15]:
y = df.groupby("customer_ID")[["target"]].max()
y

,target
customer_ID,
000201146e53cacdde1c7e9d29f4d3c46fd4d9231a3744aa39fb9c6afa79b708,0
00055add5eaee481b27e40af3f47b8b24e82c1e550f6ab010000e7685692f281,0
0005e52a3fa31b7eed49ceb576f011433ee2578833cd3f9d51c9dd9448a198ff,1
0008c2f297e1b00bf567c0d2c25f3e3b356f9a3088d2bf47aaaa724d26df8787,1
000ee46c042bfab551c28d92c93969f8a3539fe1e9fc9cd2d2e188f838f7d8ba,0
...,...
fff5d7b60546129d6a8329f6d7ae6fa18e5ebae6e2ce445f96eb933f4d62e347,0
fff7c9afcb6185ebe6152587453d9bd1eb04c458005d64c6afba25f32faa51f2,0
fffce0cbbd3dc853a970145cc1ebd44c716af633214cd0b54d18b01fbd31b626,0


<IPython.core.display.Javascript object>

In [30]:
#X = df1.groupby("customer_ID").mean().join(categorical_df)
X = df2.join(categorical_df)
X

,R_24__sum_values,R_24__median,R_24__mean,R_24__length,R_24__standard_deviation,R_24__variance,R_24__root_mean_square,R_24__maximum,R_24__absolute_maximum,R_24__minimum,...,B_38,D_114,D_116,D_117,D_120,D_126,D_63,D_64,D_66,D_68
000201146e53cacdde1c7e9d29f4d3c46fd4d9231a3744aa39fb9c6afa79b708,0.067303,0.004525,0.005177,13.0,0.002827,7.991826e-06,0.005899,0.009699,0.009699,0.000649,...,4,1,0,6,0,2,1,1,1,6
00055add5eaee481b27e40af3f47b8b24e82c1e550f6ab010000e7685692f281,0.072608,0.005156,0.005585,13.0,0.002429,5.902278e-06,0.006091,0.009010,0.009010,0.000410,...,0,0,0,3,1,1,1,3,2,4
0005e52a3fa31b7eed49ceb576f011433ee2578833cd3f9d51c9dd9448a198ff,0.033194,0.004326,0.004149,8.0,0.002699,7.284470e-06,0.004950,0.008165,0.008165,0.000209,...,3,2,2,7,2,3,1,4,2,7
0008c2f297e1b00bf567c0d2c25f3e3b356f9a3088d2bf47aaaa724d26df8787,0.067262,0.004502,0.005174,13.0,0.002472,6.109416e-06,0.005734,0.009218,0.009218,0.001211,...,6,1,0,4,1,2,1,1,2,6
000ee46c042bfab551c28d92c93969f8a3539fe1e9fc9cd2d2e188f838f7d8ba,0.066875,0.005031,0.005144,13.0,0.002926,8.561354e-06,0.005918,0.009853,0.009853,0.000882,...,1,1,0,0,0,2,1,3,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff5d7b60546129d6a8329f6d7ae6fa18e5ebae6e2ce445f96eb933f4d62e347,0.063645,0.004536,0.004896,13.0,0.002560,6.554579e-06,0.005525,0.009860,0.009860,0.000167,...,1,2,2,7,2,3,2,4,2,7
fff7c9afcb6185ebe6152587453d9bd1eb04c458005d64c6afba25f32faa51f2,0.059593,0.005046,0.004584,13.0,0.003016,9.098401e-06,0.005487,0.009918,0.009918,0.000516,...,1,1,0,5,0,2,2,4,2,6
fffce0cbbd3dc853a970145cc1ebd44c716af633214cd0b54d18b01fbd31b626,0.048996,0.002628,0.003769,13.0,0.002673,7.143774e-06,0.004620,0.008724,0.008724,0.000061,...,4,1,0,0,0,2,1,3,2,5
fffec7d7e1ca804c86f1ffdaac389c33f8039ed35bf412b12d2e3548e49d54fa,0.029304,0.005143,0.004186,7.0,0.002458,6.043732e-06,0.004855,0.007407,0.007407,0.000109,...,0,0,0,5,0,2,1,2,2,6


limit_output extension: Maximum message size of 10000 exceeded with 10779 characters

In [31]:
X_train, X_true, y_train, y_true = train_test_split(
    X, y, test_size=0.1, random_state=42
)
X_train.shape, X_true.shape, y_train.shape, y_true.shape

((26222, 1781), (2914, 1781), (26222, 1), (2914, 1))

<IPython.core.display.Javascript object>

In [32]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42
)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((23599, 1781), (2623, 1781), (23599, 1), (2623, 1))

<IPython.core.display.Javascript object>

# Train

In [33]:
model = CatBoostClassifier()
model

<IPython.core.display.Javascript object>

In [34]:
model.fit(Pool(X_train, y_train), eval_set=Pool(X_val, y_val), verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

<IPython.core.display.Javascript object>

In [35]:
model.best_score_

{'learn': {'Logloss': 0.06164538764739276},
 'validation': {'Logloss': 0.22713963990632374}}

<IPython.core.display.Javascript object>

# Test

In [36]:
y_preds = model.predict(X_true)
y_preds

array([0, 0, 0, ..., 0, 1, 0])

<IPython.core.display.Javascript object>

In [37]:
accuracy_score(y_true, y_preds)

0.8912148249828414

<IPython.core.display.Javascript object>

In [38]:
submission = pd.DataFrame(
    {
        "customer_ID": X_true.index,
        "prediction": model.predict_proba(X_true)[:, 1],
    }
).set_index("customer_ID")
submission

,prediction
customer_ID,
1d0e1bfb13c92134fb74d0dea267b8be76d39a255ccf37923e072ceba92921c4,0.016129
453d6a1185af510532421445ad192f150303f420ffc259f45f2efb41b370f123,0.025604
905a6ca1469d829e91217e0849f779478e150f7de427ba156d84c135978f6a7c,0.259911
3c30945bece00d124d711ced494cd21f204d40d7e48b72fdc5e4bc203cc2a054,0.520445
990ed4865b52d9febd9bc9a3d9847280c7902f416a1bc6f768fcfc68922be9cc,0.101763
...,...
7601f997a134164b2af9008b6b3c4b5c6df7698b31d1b45482dd48c5d9882769,0.002398
0d476719100794b3fb774411e28eca823d20f9bfc33666064e2eadf470c37967,0.108385
1e67f54a90ef89cdf63c32952884932b5a62b375c57bb58f8a32955fb096dfd2,0.005457


<IPython.core.display.Javascript object>

In [39]:
amex_metric(y_true, submission)

0.7388390671612135

<IPython.core.display.Javascript object>